#

glow-mdsol/soa-bridge-match/tree/feature/connectathon_31

In [ ]:
from pathlib import Path
import json, datetime

In [ ]:
base_path = '/Users/butterp/github/glow-mdsol/soa-bridge-match/upstream/subjects'
base_path = Path(base_path)

In [ ]:
base_path.is_dir()

True

In [ ]:
for p in base_path.glob('*'):
    print(p)

\Users\butterp\github\glow-mdsol\soa-bridge-match\upstream\subjects\LZZT_FHIR_Bundle_01-701-1015_All_Resources.json
\Users\butterp\github\glow-mdsol\soa-bridge-match\upstream\subjects\LZZT_FHIR_Bundle_01-701-1023_All_Resources.json
\Users\butterp\github\glow-mdsol\soa-bridge-match\upstream\subjects\LZZT_FHIR_Bundle_01-701-1028_All_Resources.json
\Users\butterp\github\glow-mdsol\soa-bridge-match\upstream\subjects\LZZT_FHIR_Bundle_01-701-1033_All_Resources.json
\Users\butterp\github\glow-mdsol\soa-bridge-match\upstream\subjects\LZZT_FHIR_Bundle_01-701-1034_All_Resources.json
\Users\butterp\github\glow-mdsol\soa-bridge-match\upstream\subjects\LZZT_FHIR_Bundle_01-701-1047_All_Resources.json
\Users\butterp\github\glow-mdsol\soa-bridge-match\upstream\subjects\LZZT_FHIR_Bundle_01-701-1057_All_Resources.json
\Users\butterp\github\glow-mdsol\soa-bridge-match\upstream\subjects\LZZT_FHIR_Bundle_01-701-1097_All_Resources.json
\Users\butterp\github\glow-mdsol\soa-bridge-match\upstream\subjects\LZZT

In [ ]:
with open(p) as f:
    bundle = json.load(f)

In [ ]:
def pull_attr(resouce, attr_path):
    "Pull a value from `resource` if we can find the attribute specified"
    for _attr_path in attr_path.split(' OR '):
        r, found = resouce, True
        for _attr in _attr_path.split('.'):
            if not _attr in r:
                found = False
                break
            r = r[_attr]
            if isinstance(r, list) and r:
                r = r[0] # TODO: is it OK to just pull the 1st item from the list?
        if found:
            return r

In [ ]:
def pull_resources_by_type(bundle, resource_type, required_attrs=None):
    result = []
    for entry in bundle['entry']:
        resource = entry['resource']
        if not pull_attr(resource, 'resourceType').startswith(resource_type):
            continue
        required_attrs_found = True
        if required_attrs is not None:
            for required_attr in required_attrs:
                if required_attr not in resource:
                    required_attrs_found = False
        if required_attrs_found:
            result.append(resource)
    return result

In [ ]:
# pull_resources_by_type(bundles[0], 'Medication')

In [ ]:
pull_resources_by_type(bundle, 'Patient')
# pull_resources_by_type(bundles[0], 'Encounter', ['identifier'])

[{'resourceType': 'Patient',
  'id': 'b60cb44e48594dfaaf1b07885d5b0b2d',
  'extension': [{'extension': [{'url': 'detailed',
      'valueCoding': {'system': 'urn:oid:2.16.840.1.113883.6.238',
       'code': '2106-3',
       'display': 'White'}}],
    'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-race'},
   {'extension': [{'url': 'detailed',
      'valueCoding': {'system': 'urn:oid:2.16.840.1.113883.6.238',
       'code': '2186-5',
       'display': 'Non Hispanic or Latino'}}],
    'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity'},
   {'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-birthsex',
    'valueCode': 'M'}],
  'gender': 'male',
  'birthDate': '1961-12-16',
  '_birthDate': {'fhir_comments': ['birthdate calculated from AGE = 52 and RFSTDTC = 2014-03-12 with subtraction of random integer between 0 and 365']},
  'managingOrganization': {'reference': 'Organization/b4a528955b84f584974e92d025a75d1f',
   'fhir_comments': ['mana

```
/Patient?birthdate=le2002-09-01&gender=male,female

/Encounter?reason-code:below=I20,I21,I22,I23,I24,I25&date=ge2020-09-01&date=le2021-09-31&status=finished&dischargeDisposition:not=exp

/MedicationAdministration?status=completed&effective-time=ge[Encounter-Start-Date]&
  code=http://www.nlm.nih.gov/research/umls/rxnorm|1116632,http://www.nlm.nih.gov/research/umls/rxnorm|613391,http://www.nlm.nih.gov/research/umls/rxnorm|32968,http://www.nlm.nih.gov/research/umls/rxnorm|687667,http://www.nlm.nih.gov/research/umls/rxnorm|153658
```

In [ ]:
# bundle['entry'][1]

In [ ]:
def test_inclusion_criteria(bundle):
    birth_date_cutoff = datetime.date.fromisoformat('2002-09-01')
    patients = pull_resources_by_type(bundle, 'Patient', ['birthDate', 'gender'])
    if len(patients) != 1:
        print('not one patient', len(patients))
        return False
    patient = patients[0]
    birth_date = datetime.date.fromisoformat(patient['birthDate'])
    if birth_date > birth_date_cutoff:
        print('patient is too young', birth_date)
        return False
    if patient['gender'] not in ['male', 'female']:
        print('not male or female', patient['gender'])
        return False
    # TODO: encounter date/period
#     encounters = pull_resources_by_type(bundle, 'Encounter', ['reasonCode', 'status', 'dischargeDisposition'])
#     if len(encounters) == 0:
#         print('no encounters')
#         return False
    print('med_types', set([med['resourceType'] for med in pull_resources_by_type(bundle, 'Medication')]))
    medication_count = 0
    # we don't have effectiveDateTime
    meds = pull_resources_by_type(bundle, 'MedicationStatement', ['status', 'effectivePeriod', 'medicationCodeableConcept'])
    meds = [med for med in meds if med['status'] == 'completed']
    def med_filter(med):
        system = pull_attr(med, 'medicationCodeableConcept.coding.system')
        code = pull_attr(med, 'medicationCodeableConcept.coding.code')
        if system != 'http://www.nlm.nih.gov/research/umls/rxnorm':
            print('not rxnorm')
            return False
        if code not in (['1116632','613391','32968','687667','153658']):
#             print('not a good code')
            return False
#         print(
#         pull_attr(med, 'medicationCodeableConcept.coding.system'),
#         pull_attr(med, 'medicationCodeableConcept.coding.code'))
    meds = [med for med in meds if med_filter(med)]
    medication_count += len(meds)
#     for medication_type in ['MedicationAdministration', 'MedicationStatement', 'MedicationRequest', 'MedicationDispense']: # TODO: xxx complete
#         meds = pull_resources_by_type(bundle, medication_type, ['status'])
#         medication_count += len(meds)
    if medication_count == 0:
        print('no meds')
        return False
    
    return True

In [ ]:
bundles = []
for p in base_path.glob('*'):
    with open(p) as f:
        bundle = json.load(f)
        bundles.append(bundle)
    print(p.name, test_inclusion_criteria(bundle))

med_types {'Medication', 'MedicationStatement'}
no meds
LZZT_FHIR_Bundle_01-701-1015_All_Resources.json False
med_types {'Medication', 'MedicationStatement'}
no meds
LZZT_FHIR_Bundle_01-701-1023_All_Resources.json False
med_types {'Medication', 'MedicationStatement'}
no meds
LZZT_FHIR_Bundle_01-701-1028_All_Resources.json False
med_types {'Medication', 'MedicationStatement'}
no meds
LZZT_FHIR_Bundle_01-701-1033_All_Resources.json False
med_types {'Medication', 'MedicationStatement'}
no meds
LZZT_FHIR_Bundle_01-701-1034_All_Resources.json False
med_types {'Medication', 'MedicationStatement'}
no meds
LZZT_FHIR_Bundle_01-701-1047_All_Resources.json False
not one patient 0
LZZT_FHIR_Bundle_01-701-1057_All_Resources.json False
med_types {'Medication', 'MedicationStatement'}
no meds
LZZT_FHIR_Bundle_01-701-1097_All_Resources.json False
med_types {'Medication', 'MedicationStatement'}
no meds
LZZT_FHIR_Bundle_01-701-1111_All_Resources.json False
med_types {'Medication', 'MedicationStatement'}


In [ ]:
# test_inclusion_criteria(bundle)

In [ ]:
pull_resources_by_type(bundles[0], 'MedicationStatement')

[{'resourceType': 'MedicationStatement',
  'id': 'd1e5',
  'status': 'completed',
  'medicationCodeableConcept': {'coding': [{'system': 'http://www.nlm.nih.gov/research/umls/rxnorm',
     'code': '211874',
     'display': 'ASPIRIN'}]},
  'subject': {'reference': 'Patient/897dfe86f0b710793927d8034e568ee4'},
  'effectivePeriod': {'start': '2003'},
  'dosage': [{'text': '1 TABLET ORAL PRN',
    'asNeededBoolean': True,
    'route': {'coding': [{'system': 'http://snomed.info/ct',
       'code': '26643006',
       'display': 'oral'}]},
    'doseAndRate': [{'doseQuantity': {'value': 1,
       'system': 'http://snomed.info/sct',
       'code': '428673006'}}]}]},
 {'resourceType': 'MedicationStatement',
  'id': 'd1e44',
  'status': 'completed',
  'medicationCodeableConcept': {'coding': [{'system': 'http://www.nlm.nih.gov/research/umls/rxnorm',
     'code': '211874',
     'display': 'ASPIRIN'}]},
  'subject': {'reference': 'Patient/897dfe86f0b710793927d8034e568ee4'},
  'effectivePeriod': {'star